### TO DO LIST:
<ol type="decimal">
  <li>Preparação e exploração dos dados</li>
  <li>Métricas de qualidade e validação de modelos</li>
  <li>Árvores de decisão</li>
  <li>Regressão linear e logística</li>
</ol>

# TAREFA DATASET KAGGLE

**Nota: Melhorar Descrição** 

Para esta fase, o objetivo é alcançar o melhor resultado possível com o datase da competição fornecido pelos docentes.

**1.** Importar as bibliotecas essenciais do Python para a elaboração desta tarefa

In [22]:
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**2.** Carregar o dataset para um dataframe da biblioteca Panda 

In [23]:
ds_training = pd.read_csv('docs/training_data.csv', encoding="utf-8", skipinitialspace=True)

**3.** Obtenção de informação acerca do dataset: tipos de dados das features, conteúdo do dataset e estatística

* tipos de dados das features;

In [24]:
ds_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   city_name           5000 non-null   object 
 1   magnitude_of_delay  5000 non-null   object 
 2   delay_in_seconds    5000 non-null   int64  
 3   affected_roads      4915 non-null   object 
 4   record_date         5000 non-null   object 
 5   luminosity          5000 non-null   object 
 6   avg_temperature     5000 non-null   float64
 7   avg_atm_pressure    5000 non-null   float64
 8   avg_humidity        5000 non-null   float64
 9   avg_wind_speed      5000 non-null   float64
 10  avg_precipitation   5000 non-null   float64
 11  avg_rain            5000 non-null   object 
 12  incidents           5000 non-null   object 
dtypes: float64(5), int64(1), object(7)
memory usage: 507.9+ KB


* conteúdo do dataset;

In [25]:
ds_training.head()

,city_name,magnitude_of_delay,delay_in_seconds,affected_roads,record_date,luminosity,avg_temperature,avg_atm_pressure,avg_humidity,avg_wind_speed,avg_precipitation,avg_rain,incidents
0,Guimaraes,UNDEFINED,0,",",2021-03-15 23:00,DARK,12.0,1013.0,70.0,1.0,0.0,Sem Chuva,None
1,Guimaraes,UNDEFINED,385,"N101,",2021-12-25 18:00,DARK,12.0,1007.0,91.0,1.0,0.0,Sem Chuva,None
2,Guimaraes,UNDEFINED,69,",",2021-03-12 15:00,LIGHT,14.0,1025.0,64.0,0.0,0.0,Sem Chuva,Low
3,Guimaraes,MAJOR,2297,"N101,R206,N105,N101,N101,N101,N101,N101,N101,N...",2021-09-29 09:00,LIGHT,15.0,1028.0,75.0,1.0,0.0,Sem Chuva,Very_High
4,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101,",2021-06-13 11:00,LIGHT,27.0,1020.0,52.0,1.0,0.0,Sem Chuva,High


* estatística;

In [26]:
ds_training.describe()

,delay_in_seconds,avg_temperature,avg_atm_pressure,avg_humidity,avg_wind_speed,avg_precipitation
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0
mean,560.567000,14.583000,1018.145000,74.455000,1.253500,0.0
std,1686.859581,4.820514,5.174372,17.204638,1.269847,0.0
min,0.000000,1.000000,997.000000,6.000000,0.000000,0.0
25%,0.000000,11.000000,1015.000000,63.000000,0.000000,0.0
50%,0.000000,14.000000,1019.000000,78.000000,1.000000,0.0
75%,234.000000,18.000000,1022.000000,90.000000,2.000000,0.0
max,31083.000000,35.000000,1032.000000,100.000000,10.000000,0.0


**4.** Preparação dos dados

In [27]:
#Remoção de "avg_precipitation" - tem informação incompleta
ds_training = ds_training.drop('avg_precipitation', axis=1)



In [28]:
#separar coluna de record_date em ano, mes, dia, hora e minuto
ds_training['record_date'] = pd.to_datetime(ds_training['record_date'],format='%Y-%m-%d %H:%M', errors='coerce')
assert ds_training['record_date'].isnull().sum() == 0,'missing record date'

ds_training['record_date_year'] = ds_training['record_date'].dt.year
ds_training['record_date_month'] = ds_training['record_date'].dt.month
ds_training['record_date_day'] = ds_training['record_date'].dt.day
ds_training['record_date_hour'] = ds_training['record_date'].dt.hour
ds_training['record_date_minute'] = ds_training['record_date'].dt.minute
ds_training = ds_training.drop('record_date', axis=1)

ds_training.head()

,city_name,magnitude_of_delay,delay_in_seconds,affected_roads,luminosity,avg_temperature,avg_atm_pressure,avg_humidity,avg_wind_speed,avg_rain,incidents,record_date_year,record_date_month,record_date_day,record_date_hour,record_date_minute
0,Guimaraes,UNDEFINED,0,",",DARK,12.0,1013.0,70.0,1.0,Sem Chuva,None,2021,3,15,23,0
1,Guimaraes,UNDEFINED,385,"N101,",DARK,12.0,1007.0,91.0,1.0,Sem Chuva,None,2021,12,25,18,0
2,Guimaraes,UNDEFINED,69,",",LIGHT,14.0,1025.0,64.0,0.0,Sem Chuva,Low,2021,3,12,15,0
3,Guimaraes,MAJOR,2297,"N101,R206,N105,N101,N101,N101,N101,N101,N101,N...",LIGHT,15.0,1028.0,75.0,1.0,Sem Chuva,Very_High,2021,9,29,9,0
4,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101,",LIGHT,27.0,1020.0,52.0,1.0,Sem Chuva,High,2021,6,13,11,0


In [29]:
#analisar numero de valores unicos de cada atributo
print(ds_training.nunique())
print()
for column in ds_training:
    print(f'Column {column}:')
    print(ds_training[column].value_counts)
    print()

city_name                1
magnitude_of_delay       3
delay_in_seconds      1186
affected_roads         678
luminosity               3
avg_temperature         35
avg_atm_pressure        36
avg_humidity            83
avg_wind_speed          11
avg_rain                 4
incidents                5
record_date_year         1
record_date_month       11
record_date_day         31
record_date_hour        24
record_date_minute       1
dtype: int64

Column city_name:
<bound method IndexOpsMixin.value_counts of 0       Guimaraes
1       Guimaraes
2       Guimaraes
3       Guimaraes
4       Guimaraes
          ...    
4995    Guimaraes
4996    Guimaraes
4997    Guimaraes
4998    Guimaraes
4999    Guimaraes
Name: city_name, Length: 5000, dtype: object>

Column magnitude_of_delay:
<bound method IndexOpsMixin.value_counts of 0       UNDEFINED
1       UNDEFINED
2       UNDEFINED
3           MAJOR
4       UNDEFINED
          ...    
4995    UNDEFINED
4996    UNDEFINED
4997    UNDEFINED
4998    UNDEFI

In [30]:
#como apenas existe um valor para "city_name", esta coluna pode ser removida
ds_training = ds_training.drop('city_name', axis=1)

#o mesmo para o ano e minuto
ds_training = ds_training.drop('record_date_year', axis=1)
ds_training = ds_training.drop('record_date_minute', axis=1)

In [31]:
#tornar magnitude of delay numerico
ds_training['magnitude_of_delay'].unique()

array(['UNDEFINED', 'MAJOR', 'MODERATE'], dtype=object)

In [32]:
replace_map = {'magnitude_of_delay': {'UNDEFINED':0 ,'MODERATE': 1, 'MAJOR': 2}}
ds_training.replace(replace_map,inplace=True)
ds_training.head()

,magnitude_of_delay,delay_in_seconds,affected_roads,luminosity,avg_temperature,avg_atm_pressure,avg_humidity,avg_wind_speed,avg_rain,incidents,record_date_month,record_date_day,record_date_hour
0,0,0,",",DARK,12.0,1013.0,70.0,1.0,Sem Chuva,None,3,15,23
1,0,385,"N101,",DARK,12.0,1007.0,91.0,1.0,Sem Chuva,None,12,25,18
2,0,69,",",LIGHT,14.0,1025.0,64.0,0.0,Sem Chuva,Low,3,12,15
3,2,2297,"N101,R206,N105,N101,N101,N101,N101,N101,N101,N...",LIGHT,15.0,1028.0,75.0,1.0,Sem Chuva,Very_High,9,29,9
4,0,0,"N101,N101,N101,N101,N101,",LIGHT,27.0,1020.0,52.0,1.0,Sem Chuva,High,6,13,11


In [42]:
#tratar affected_roads
ds_training['affected_roads'].fillna(',',inplace=True) #tratar de missing values
ds_training.info()

affected_roads = {}
for row in ds_training['affected_roads']:
    roads = list(dict.fromkeys(row.split(','))) #separa as ruas e retira repetidas
    for road in roads:
        affected_roads[road] = []

for row in ds_training['affected_roads']:
    roads = list(dict.fromkeys(row.split(',')))
    for road in test:
        if road in roads:
            affected_roads[road].append(1)
        else:
            affected_roads[road].append(0)

        
affected_roads = pd.DataFrame.from_dict(affected_roads)
ds_training = ds_training.drop('affected_roads',axis=1)

for column in affected_roads:
    ds_training[column] = affected_roads[column]
    
ds_training

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   magnitude_of_delay  5000 non-null   int64  
 1   delay_in_seconds    5000 non-null   int64  
 2   affected_roads      5000 non-null   object 
 3   luminosity          5000 non-null   object 
 4   avg_temperature     5000 non-null   float64
 5   avg_atm_pressure    5000 non-null   float64
 6   avg_humidity        5000 non-null   float64
 7   avg_wind_speed      5000 non-null   float64
 8   avg_rain            5000 non-null   object 
 9   incidents           5000 non-null   object 
 10  record_date_month   5000 non-null   int64  
 11  record_date_day     5000 non-null   int64  
 12  record_date_hour    5000 non-null   int64  
dtypes: float64(4), int64(5), object(4)
memory usage: 507.9+ KB


,magnitude_of_delay,delay_in_seconds,luminosity,avg_temperature,avg_atm_pressure,avg_humidity,avg_wind_speed,avg_rain,incidents,record_date_month,...,N101,R206,N105,N206,N309,IC5,N310,N207-4,IC5 - N206,EM579
0,0,0,DARK,12.0,1013.0,70.0,1.0,Sem Chuva,None,3,...,0,0,0,0,0,0,0,0,0,0
1,0,385,DARK,12.0,1007.0,91.0,1.0,Sem Chuva,None,12,...,1,0,0,0,0,0,0,0,0,0
2,0,69,LIGHT,14.0,1025.0,64.0,0.0,Sem Chuva,Low,3,...,0,0,0,0,0,0,0,0,0,0
3,2,2297,LIGHT,15.0,1028.0,75.0,1.0,Sem Chuva,Very_High,9,...,1,1,1,0,0,0,0,0,0,0
4,0,0,LIGHT,27.0,1020.0,52.0,1.0,Sem Chuva,High,6,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,DARK,13.0,1021.0,56.0,0.0,Sem Chuva,High,4,...,1,0,0,0,0,0,0,0,0,0
4996,0,0,LIGHT,19.0,1017.0,68.0,0.0,Sem Chuva,None,7,...,1,0,0,0,0,0,0,0,0,0
4997,0,0,DARK,13.0,1023.0,47.0,2.0,Sem Chuva,None,3,...,0,0,0,0,0,0,0,0,0,0
4998,0,0,DARK,12.0,1014.0,93.0,2.0,Sem Chuva,None,11,...,1,0,0,0,0,0,0,0,0,0


In [44]:
#tornar luminosity numerico
ds_training['luminosity'].unique()

array(['DARK', 'LIGHT', 'LOW_LIGHT'], dtype=object)

In [45]:
replace_map = {'luminosity': {'LOW_LIGHT':0 ,'LIGHT': 1, 'DARK': 2}}
ds_training.replace(replace_map,inplace=True)
ds_training.head()

,magnitude_of_delay,delay_in_seconds,luminosity,avg_temperature,avg_atm_pressure,avg_humidity,avg_wind_speed,avg_rain,incidents,record_date_month,...,N101,R206,N105,N206,N309,IC5,N310,N207-4,IC5 - N206,EM579
0,0,0,2,12.0,1013.0,70.0,1.0,Sem Chuva,None,3,...,0,0,0,0,0,0,0,0,0,0
1,0,385,2,12.0,1007.0,91.0,1.0,Sem Chuva,None,12,...,1,0,0,0,0,0,0,0,0,0
2,0,69,1,14.0,1025.0,64.0,0.0,Sem Chuva,Low,3,...,0,0,0,0,0,0,0,0,0,0
3,2,2297,1,15.0,1028.0,75.0,1.0,Sem Chuva,Very_High,9,...,1,1,1,0,0,0,0,0,0,0
4,0,0,1,27.0,1020.0,52.0,1.0,Sem Chuva,High,6,...,1,0,0,0,0,0,0,0,0,0


In [47]:
#tornar avg rain numerico
ds_training['avg_rain'].unique()

array(['Sem Chuva', 'chuva moderada', 'chuva fraca', 'chuva forte'],
      dtype=object)

In [48]:
replace_map = {'avg_rain': {'Sem Chuva':0 ,'chuva fraca': 1, 'chuva moderada': 2,'chuva forte' : 3}}
ds_training.replace(replace_map,inplace=True)
ds_training.head()

,magnitude_of_delay,delay_in_seconds,luminosity,avg_temperature,avg_atm_pressure,avg_humidity,avg_wind_speed,avg_rain,incidents,record_date_month,...,N101,R206,N105,N206,N309,IC5,N310,N207-4,IC5 - N206,EM579
0,0,0,2,12.0,1013.0,70.0,1.0,0,None,3,...,0,0,0,0,0,0,0,0,0,0
1,0,385,2,12.0,1007.0,91.0,1.0,0,None,12,...,1,0,0,0,0,0,0,0,0,0
2,0,69,1,14.0,1025.0,64.0,0.0,0,Low,3,...,0,0,0,0,0,0,0,0,0,0
3,2,2297,1,15.0,1028.0,75.0,1.0,0,Very_High,9,...,1,1,1,0,0,0,0,0,0,0
4,0,0,1,27.0,1020.0,52.0,1.0,0,High,6,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
#tornar incidents em numerico
replace_map = {'incidents': {'None':0 ,'Low': 1, 'Medium': 2, 'High': 3, 'Very_High': 4}}
ds_training.replace(replace_map,inplace=True)
ds_training.head()

**5.** Redes neuronais

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler

RANDOM_SEED = 2021

print("TensorFlow version:", tf.__version__)

In [ ]:
x = ds_training.drop('incidents',axis=1)
y = ds_training[['incidents']]